In [2]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

Mounted at /content/gdrive


In [3]:
import os
from pathlib import Path

folder = "colab/pytorch"
project_dir = "Auto_Sklearn"

base_path = Path("/content/gdrive/My Drive/")
project_path = base_path / folder / project_dir
os.chdir(project_path)
for x in list(project_path.glob("*")):
    if x.is_dir():
        dir_name = str(x.relative_to(project_path))
        os.rename(dir_name, dir_name.split(" ", 1)[0])
print(f"현재 디렉토리 위치: {os.getcwd()}")

현재 디렉토리 위치: /content/gdrive/My Drive/colab/pytorch/Auto_Sklearn


In [4]:
import numpy as np
import pandas as pd

In [5]:
data = pd.read_csv('/content/gdrive/My Drive/colab/pytorch/Auto_Sklearn/Data/data.csv')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [7]:
data = data.drop(columns=['Unnamed: 32'])

In [8]:
data.diagnosis.value_counts(normalize=True)

B    0.627417
M    0.372583
Name: diagnosis, dtype: float64

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data[[col for col in data.columns if col not in ['id','diagnosis']]],
                                                    data['diagnosis'], test_size = 0.33)

In [10]:
%pip install auto-sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 5.8 MB/s 
     |████████████████████████████████| 22.3 MB 1.1 MB/s 
     |████████████████████████████████| 4.3 MB 45.7 MB/s 
     |████████████████████████████████| 4.4 MB 34.4 MB/s 
     |████████████████████████████████| 260 kB 66.4 MB/s 
     |████████████████████████████████| 46 kB 4.0 MB/s 
  Created wheel for auto-sklearn: filename=auto_sklearn-0.14.7-py3-none-any.whl size=6602873 sha256=50e589392ea28508b892282e26c9180b62970bb9d6284ad91d311ad93da61295
  Stored in directory: /root/.cache/pip/wheels/ba/43/5c/2fbe6fd19e3af314cbc4aa808378068d8ddd6792064f4a2448
  Created wheel for pynisher: filename=pynisher-0.6.4-py3-none-any.whl size=7043 sha256=519edc102741b27c3f0561e79734f98856487698478ddccd8875d4082ca27125
  Stored in directory: /root/.cache/pip/wheels/42/71/95/7555ec3253e1ba8add72ae5febf1b015d297f3b73ba296d6f6
  Created wheel for smac

In [12]:
import autosklearn.classification

In [13]:
help(autosklearn.classification.AutoSklearnClassifier)

Help on class AutoSklearnClassifier in module autosklearn.estimators:

class AutoSklearnClassifier(AutoSklearnEstimator, sklearn.base.ClassifierMixin)
 |  AutoSklearnClassifier(time_left_for_this_task=3600, per_run_time_limit=None, initial_configurations_via_metalearning=25, ensemble_size: int = 50, ensemble_nbest=50, max_models_on_disc=50, seed=1, memory_limit=3072, include: Union[Dict[str, List[str]], NoneType] = None, exclude: Union[Dict[str, List[str]], NoneType] = None, resampling_strategy='holdout', resampling_strategy_arguments=None, tmp_folder=None, delete_tmp_folder_after_terminate=True, n_jobs: Union[int, NoneType] = None, dask_client: Union[distributed.client.Client, NoneType] = None, disable_evaluator_output=False, get_smac_object_callback=None, smac_scenario_args=None, logging_config=None, metadata_directory=None, metric=None, scoring_functions: Union[List[autosklearn.metrics.Scorer], NoneType] = None, load_models: bool = True, get_trials_callback=None)
 |  
 |  This class

In [14]:
%%time
automl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task = 300, tmp_folder = "./log/") # 5분 탐색을 하도록 해보자
automl.fit(x_train, y_train)

CPU times: user 1min 39s, sys: 5.13 s, total: 1min 44s
Wall time: 4min 55s


AutoSklearnClassifier(per_run_time_limit=30, time_left_for_this_task=300,
                      tmp_folder='./log/')

In [15]:
automl.leaderboard()

,rank,ensemble_weight,type,cost,duration
model_id,,,,,
2,1,0.04,random_forest,0.007937,3.160889
3,2,0.04,mlp,0.007937,2.244423
6,3,0.02,random_forest,0.007937,2.335724
7,4,0.02,extra_trees,0.007937,1.832102
64,5,0.06,random_forest,0.007937,2.104560
21,6,0.04,random_forest,0.007937,2.338849
41,7,0.02,random_forest,0.007937,3.375219
55,8,0.04,random_forest,0.007937,6.106305
54,9,0.06,random_forest,0.007937,2.986759


In [16]:
print(automl.sprint_statistics())

auto-sklearn results:
  Dataset name: 2091ce18-2104-11ed-8047-0242ac1c0002
  Metric: accuracy
  Best validation score: 0.992063
  Number of target algorithm runs: 64
  Number of successful target algorithm runs: 64
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 0
  Number of target algorithms that exceeded the memory limit: 0



In [17]:
print(len(automl.get_models_with_weights()))
for info in automl.get_models_with_weights():
    print("\n")
    print("model, weight : {}\n{}".format(info[0], "-" * 10 + "\n" + str(info[1])))

32


model, weight : 0.06
----------
SimpleClassificationPipeline({'balancing:strategy': 'weighting', 'classifier:__choice__': 'random_forest', 'data_preprocessor:__choice__': 'feature_type', 'feature_preprocessor:__choice__': 'polynomial', 'classifier:random_forest:bootstrap': 'True', 'classifier:random_forest:criterion': 'entropy', 'classifier:random_forest:max_depth': 'None', 'classifier:random_forest:max_features': 0.5182129405427335, 'classifier:random_forest:max_leaf_nodes': 'None', 'classifier:random_forest:min_impurity_decrease': 0.0, 'classifier:random_forest:min_samples_leaf': 1, 'classifier:random_forest:min_samples_split': 4, 'classifier:random_forest:min_weight_fraction_leaf': 0.0, 'data_preprocessor:feature_type:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding', 'data_preprocessor:feature_type:categorical_transformer:category_coalescence:__choice__': 'no_coalescense', 'data_preprocessor:feature_type:numerical_transformer:imputation:strategy': 'm

In [18]:
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix

In [19]:
model = automl

print(y_test.reset_index().groupby('diagnosis').size())
y_predict = model.predict(x_test)
y_predict_score = model.predict_proba(x_test)[:, 1]

print("Accuracy : {}, AUC : {}, Precision : {}, Recall : {}".format(
    accuracy_score(y_test, y_predict),
    roc_auc_score(y_test, y_predict_score),
    precision_score(y_test, y_predict, pos_label='B'),
    recall_score(y_test, y_predict, pos_label='B')
))


confusion_matrix(y_test, y_predict)

diagnosis
B    112
M     76
dtype: int64
Accuracy : 0.973404255319149, AUC : 0.9967105263157895, Precision : 0.9652173913043478, Recall : 0.9910714285714286


array([[111,   1],
       [  4,  72]])